In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [3]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [4]:
#save tfr'd data
def save_tfr_data(pp_num, root_dir, erp_path, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [5]:
#load tfr'd data
def load_tfr_data(pp_num, root_dir, erp_path, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [ ]:
#save average tfr per participant for each condition
#morlet wavelet convolution for each participant, according to trial types - POWER

# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# for pcond in range(0, len(perturb_conds)):
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         #average epochs within participant
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = True
#         )
        
#         save_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, data = power, task = perturb_conds[pcond], output = "power")

In [6]:
# load in tfr object, ensure to baseline correct; (tfr shape is channels, freqs, timepts)
# narrow down to channels 
# narrow down to frequencies and timepts
# take mean of alpha or beta ranges (freqs)
# take mean across electrodes (channels)

perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']


# channels = ['Fpz', 'AFz', #frontal (cognitive)
#             'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
#             'Cz', 'C3', 'C4', #motor execution
#             'Pz', 'P3', 'P4', #parietal areas
#             'Oz', 'O1', 'O2'] #visual areas

# FRONTAL CHANNELS
channels = ['Fpz', 'AFz', 'Fz', 'F3', 'F4']
# BASELINE FOR FEEDBACK ONSET
baseline_t = (-1.5, -1.2)

# ALPHA
freq_lower = 8
freq_upper = 13

alpha_frontal_EarlyLateAligned = []
alpha_frontal_EarlyRot = []
alpha_frontal_LateRot = []
alpha_frontal_EarlyMir = []
alpha_frontal_LateMir = []
alpha_frontal_EarlyRdm = []
alpha_frontal_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    allppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        # apply baseline correction
        dat = dat[0].apply_baseline(baseline_t)
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(350,701))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        allppdat.append(ppdat)
        
    if pcond == 0:
        alpha_frontal_EarlyLateAligned.append(allppdat)
    elif pcond == 1:
        alpha_frontal_EarlyRot.append(allppdat)
    elif pcond == 2:
        alpha_frontal_LateRot.append(allppdat)
    elif pcond == 3:
        alpha_frontal_EarlyMir.append(allppdat)
    elif pcond == 4:
        alpha_frontal_LateMir.append(allppdat)
    elif pcond == 5:
        alpha_frontal_EarlyRdm.append(allppdat)
    elif pcond == 6:
        alpha_frontal_LateRdm.append(allppdat)

# BETA
freq_lower = 13
freq_upper = 35

beta_frontal_EarlyLateAligned = []
beta_frontal_EarlyRot = []
beta_frontal_LateRot = []
beta_frontal_EarlyMir = []
beta_frontal_LateMir = []
beta_frontal_EarlyRdm = []
beta_frontal_LateRdm = []

for pcond in range(0, len(perturb_conds)):
    allppdat = []
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "power")
        # apply baseline correction
        dat = dat[0].apply_baseline(baseline_t)
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(350,701))
        npdat = npdat[:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 1) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 0) # calculate mean across channels
        
        allppdat.append(ppdat)
        
    if pcond == 0:
        beta_frontal_EarlyLateAligned.append(allppdat)
    elif pcond == 1:
        beta_frontal_EarlyRot.append(allppdat)
    elif pcond == 2:
        beta_frontal_LateRot.append(allppdat)
    elif pcond == 3:
        beta_frontal_EarlyMir.append(allppdat)
    elif pcond == 4:
        beta_frontal_LateMir.append(allppdat)
    elif pcond == 5:
        beta_frontal_EarlyRdm.append(allppdat)
    elif pcond == 6:
        beta_frontal_LateRdm.append(allppdat)
    

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/frn\p000_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p001/frn\p001_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p002/frn\p002_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p003/frn\p003_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p004/frn\p004_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p005/frn\p005_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p006/frn\p006_early_late_aligned_power-tfr.h5 ...
Applying baseline correctio

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/frn\p027_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p028/frn\p028_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p029/frn\p029_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p030/frn\p030_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p031/frn\p031_early_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/frn\p000_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p001/frn\p001_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading 

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p024/frn\p024_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p025/frn\p025_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p026/frn\p026_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/frn\p027_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p028/frn\p028_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p029/frn\p029_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p030/frn\p030_early_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Readin

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p021/frn\p021_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p022/frn\p022_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p023/frn\p023_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p024/frn\p024_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p025/frn\p025_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p026/frn\p026_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p027/frn\p027_early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Readin

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p017/frn\p017_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p018/frn\p018_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p019/frn\p019_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p020/frn\p020_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p021/frn\p021_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p022/frn\p022_early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p023/frn\p023_early_late_aligned_po

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p013/frn\p013_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p014/frn\p014_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p015/frn\p015_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p016/frn\p016_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p017/frn\p017_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p018/frn\p018_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p019/frn\p019_late_rot_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Do

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p010/frn\p010_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p011/frn\p011_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p012/frn\p012_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p013/frn\p013_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p014/frn\p014_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p015/frn\p015_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p016/frn\p016_late_mir_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Do

Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p007/frn\p007_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p008/frn\p008_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p009/frn\p009_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p010/frn\p010_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p011/frn\p011_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p012/frn\p012_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p013/frn\p013_late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: mean)
Reading F:/Do

In [ ]:
# After transforming the TFR data into a single signal representative of frequency and channels
# we compare the two signals: same as taking their difference (signal1 minus signal2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [ ]:
# compare each condition to aligned (for both alpha and beta)
# subtract aligned, then compare early vs late (for both alpha and beta)
# subtract early from late, then compare perturbations (both alpha and beta)
# repeat for other channels (ROIs)

In [ ]:
#plot average tfr for each of perturbation conditions/ trial types
if erps == 'frn':
    baseline_t = [-1.5, -1.2]
elif erps == 'lrp':
    baseline_t = [-1.3, -1.0]
perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']
channels = ['Fpz', 'AFz', #frontal (cognitive)
            'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
            'Cz', 'C3', 'C4', #motor execution
            'Pz', 'P3', 'P4', #parietal areas
            'Oz', 'O1', 'O2'] #visual areas
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, baseline=(baseline_t[0], baseline_t[1]), mode="logratio", combine='mean', tmin=0, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.2, vmax = 0.2)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, baseline=(baseline_t[0], baseline_t[1]), mode="logratio", combine='mean', tmin=-1.5, tmax=0, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.2, vmax = 0.2)
        
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power")

In [ ]:
# # subtract aligned from each perturbation condition (e.g. early_rot minus aligned)
# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# if erps == 'frn':
#     baseline_t = [-1.5, -1.2]
# elif erps == 'lrp':
#     baseline_t = [-1.3, -1.0]

# early_rot = []
# late_rot = []
# early_mir = []
# late_mir = []
# early_rdm = []
# late_rdm = []
    
# for pp in participants:
#     for pcond in range(0, len(perturb_conds)):
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = False
#         )
        
#         if pcond == 0:
#             aln_power = power.average()
#             aln_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 1:
#             early_rot_power = power.average()
#             early_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 2:
#             late_rot_power = power.average()
#             late_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 3:
#             early_mir_power = power.average()
#             early_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 4:
#             late_mir_power = power.average()
#             late_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 5:
#             early_rdm_power = power.average()
#             early_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 6:
#             late_rdm_power = power.average()
#             late_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
            
#     early_rot_contrast = early_rot_power - aln_power
#     late_rot_contrast = late_rot_power - aln_power
#     early_mir_contrast = early_mir_power - aln_power
#     late_mir_contrast = late_mir_power - aln_power
#     early_rdm_contrast = early_rdm_power - aln_power
#     late_rdm_contrast = late_rdm_power - aln_power
    
#     early_rot.append(early_rot_contrast)
#     late_rot.append(late_rot_contrast)
#     early_mir.append(early_mir_contrast)
#     late_mir.append(late_mir_contrast)
#     early_rdm.append(early_rdm_contrast)
#     late_rdm.append(late_rdm_contrast)
    

# #grand average of tfr contrasts across participants    
# alldat = mne.grand_average(early_rot, interpolate_bads=True, drop_bads=True)
# #save tfr object
# output_tfr_data(root_dir = root, data = alldat, task = 'early_rot_contrast', output = "power")
 
# alldat = mne.grand_average(late_rot, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'late_rot_contrast', output = "power")

# alldat = mne.grand_average(early_mir, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'early_mir_contrast', output = "power")

# alldat = mne.grand_average(late_mir, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'late_mir_contrast', output = "power")

# alldat = mne.grand_average(early_rdm, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'early_rdm_contrast', output = "power")

# alldat = mne.grand_average(late_rdm, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'late_rdm_contrast', output = "power")



In [ ]:
#plot ALIGNED SUBTRACTED average tfr  contrasts for each of perturbation conditions/ trial types
# no baselining needed, since this was already done before subtraction
perturb_conds = ['early_rot_contrast', 'late_rot_contrast', 'early_mir_contrast', 'late_mir_contrast', 'early_rdm_contrast', 'late_rdm_contrast']
channels = ['Fpz', 'AFz', #frontal (cognitive)
            'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
            'Cz', 'C3', 'C4', #motor execution
            'Pz', 'P3', 'P4', #parietal areas
            'Oz', 'O1', 'O2'] #visual areas
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, combine='mean', tmin=0, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.1, vmax = 0.1)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=0, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.1, vmax = 0.1)
        
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power")

In [ ]:
# # subtract EARLY from LATE for each perturbation type (e.g. late_rot minus early_rot)
# # subtract baseline aligned for each participant, then subtract early from late
# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# if erps == 'frn':
#     baseline_t = [-1.5, -1.2]
# elif erps == 'lrp':
#     baseline_t = [-1.3, -1.0]

# rot = []
# mir = []
# rdm = []

    
# for pp in participants:
#     for pcond in range(0, len(perturb_conds)):
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = False
#         )
        
#         if pcond == 0:
#             aln_power = power.average()
#             aln_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 1:
#             early_rot_power = power.average()
#             early_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 2:
#             late_rot_power = power.average()
#             late_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 3:
#             early_mir_power = power.average()
#             early_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 4:
#             late_mir_power = power.average()
#             late_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 5:
#             early_rdm_power = power.average()
#             early_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 6:
#             late_rdm_power = power.average()
#             late_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
            
#     early_rot_contrast = early_rot_power - aln_power
#     late_rot_contrast = late_rot_power - aln_power
#     early_mir_contrast = early_mir_power - aln_power
#     late_mir_contrast = late_mir_power - aln_power
#     early_rdm_contrast = early_rdm_power - aln_power
#     late_rdm_contrast = late_rdm_power - aln_power
    
#     rot_contrast = late_rot_contrast - early_rot_contrast
#     mir_contrast = late_mir_contrast - early_mir_contrast
#     rdm_contrast = late_rdm_contrast - early_rdm_contrast
#     #print(late_rot_contrast, early_rot_contrast, rot_contrast)
#     rot.append(rot_contrast)
#     mir.append(mir_contrast)
#     rdm.append(rdm_contrast)
    

# #grand average of tfr contrasts across participants    
# alldat = mne.grand_average(rot, interpolate_bads=True, drop_bads=True)
# #save tfr object
# output_tfr_data(root_dir = root, data = alldat, task = 'EL_rot', output = "power")
 
# alldat = mne.grand_average(mir, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'EL_mir', output = "power")

# alldat = mne.grand_average(rdm, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'EL_rdm', output = "power")



In [ ]:
#plot EARLY and LATE SUBTRACTED average tfr  contrasts for each of perturbation conditions/ trial types
# no baselining needed, since this was already done before subtraction
perturb_conds = ['EL_rot', 'EL_mir', 'EL_rdm']
channels = ['Fpz', 'AFz', #frontal (cognitive)
            'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
            'Cz', 'C3', 'C4', #motor execution
            'Pz', 'P3', 'P4', #parietal areas
            'Oz', 'O1', 'O2'] #visual areas
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, combine='mean', tmin=0, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.08, vmax = 0.08)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=0, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.08, vmax = 0.08)
    
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power")

In [ ]:
# convert EARLY and LATE SUBTRACTED TFR to CSV
perturb_conds = ['EL_rot', 'EL_mir', 'EL_rdm']

for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    df = data[0].to_data_frame()
    df = df[['time', 'freq', 'Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz', 'Cz', 'C3', 'C4', 'Pz', 'P3', 'P4', 'Oz', 'O1', 'O2']]
    df_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', '%s_power_%s.csv' % (perturb_conds[pcond], erps))
    df.to_csv(df_filename)
    

In [ ]:
# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# for pcond in range(0, len(perturb_conds)):
#     pp_power = []
#     pp=0
#     data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#     data.compute_psd(fmin=2.0, fmax=100).plot(average=True, picks="data", exclude="bads")
# #     data.compute_psd().plot_topomap(ch_type="eeg", normalize=False, contours=0)